In [20]:
!ls /workspace/results/

df_model.onnx	       quartznet_hls_cleaned.onnx	       syn_v0  syn_v5
quantized_input.npy    quartznet_hls_cleaned_partitioned.onnx  syn_v1  syn_v6
quantized_output.npy   single_node_models		       syn_v2  syn_v7
quartznet_export.onnx  syn_end2end			       syn_v3
quartznet_hls.onnx     syn_low_folding			       syn_v4


In [21]:
from finn.util.visualization import showInNetron

#showInNetron("/workspace/results/quartznet_hls_cleaned.onnx")
showInNetron("/workspace/results/quartznet_export.onnx")

Serving '/workspace/results/quartznet_export.onnx' at http://0.0.0.0:8081


# Compare model

In [41]:
import numpy as np
from finn.core.modelwrapper import ModelWrapper
from finn.util.basic import gen_finn_dt_tensor
import finn.core.onnx_exec as oxe
import time

t1 = time.perf_counter()

################################################################################################
####
#### MODEL 1
####
model_1 = ModelWrapper("/workspace/results/quartznet_export.onnx")
#model_1 = ModelWrapper("/tmp/quartznet_export.onnx")

#### MODEL 1
# Create input data
input0_tensor_name = model_1.graph.input[0].name

## Change input...
#input_val = np.load("brevitas_reference/end2end_quartznet_input_quantized.npy")
#input_val = input_val[:,:,0:256].astype(np.float32)
input_val = np.load("/workspace/results/librispeech_data/input_sample_1.npy")

input_dict = {}
input_dict[input0_tensor_name] = input_val
output0_tensor_name = model_1.graph.output[0].name

expected_m1_dict = oxe.execute_onnx(model_1, input_dict, return_full_exec_context = False)
expected_m1 = expected_m1_dict[output0_tensor_name]
################################################################################################

t2 = time.perf_counter() - t1
print("Elapsed time: {}".format(t2))

Elapsed time: 346.10875359899364


In [42]:
golden_output = np.load("/workspace/results/librispeech_data/output_sample_1.npy")

print(np.shape(golden_output))
print(np.shape(expected_m1))

for idx, el in enumerate(golden_output[0]):
    print("{}\t{}".format(el, expected_m1[0][idx]))

(1, 128)
(1, 128)
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
20	20
8	8
28	5
5	28
28	28
28	0
0	0
0	28
19	19
28	28
16	16
28	28
28	28
28	28
9	9
14	28
28	28
28	28
28	28
0	0
0	0
23	23
28	28
1	1
19	19
19	19
28	28
0	28
0	0
28	28
21	1
28	28
28	28
28	28
16	16
28	28
28	28
28	15
15	18
18	28
28	28
28	28
28	28
0	0
0	28
9	28
14	14
0	0
0	0
0	0
20	20
8	8
5	5
28	0
0	0
0	28
28	28
28	13
2	28
28	28
28	28
28	15
5	28
28	28
28	4
28	28
4	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28


In [43]:
labels = [" ", "a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m",
         "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z", "'"]
    
def __ctc_decoder_predictions_tensor(tensor, labels):
    """
    Decodes a sequence of labels to words
    """
    blank_id = len(labels)
    hypotheses = []
    labels_map = dict([(i, labels[i]) for i in range(len(labels))])
    decoded_prediction = []
    previous = len(labels)  # id of a blank symbol
    prediction = tensor
    for p in prediction:
        if (p != previous or previous == blank_id) and p != blank_id:
            decoded_prediction.append(p)
        previous = p
    hypothesis = ''.join([labels_map[c] for c in decoded_prediction])
    hypotheses.append(hypothesis)
    return hypotheses

print(__ctc_decoder_predictions_tensor(golden_output[0], labels))
print(__ctc_decoder_predictions_tensor(expected_m1[0], labels))

['the spin was upor in the bed']
['the spi was apor n the mod']


# Compare outputs test_brevitas_quartznet

In [48]:
import numpy as np

produced = np.load("/tmp/produced.npy")
expected = np.load("/tmp/expected.npy")

for idx, el in enumerate(expected[0]):
    print("{}\t{}".format(el, produced[0][idx]))

    
labels = [" ", "a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m",
         "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z", "'"]
    
def __ctc_decoder_predictions_tensor(tensor, labels):
    """
    Decodes a sequence of labels to words
    """
    blank_id = len(labels)
    hypotheses = []
    labels_map = dict([(i, labels[i]) for i in range(len(labels))])
    decoded_prediction = []
    previous = len(labels)  # id of a blank symbol
    prediction = tensor
    for p in prediction:
        if (p != previous or previous == blank_id) and p != blank_id:
            decoded_prediction.append(p)
        previous = p
    hypothesis = ''.join([labels_map[c] for c in decoded_prediction])
    hypotheses.append(hypothesis)
    return hypotheses

print(__ctc_decoder_predictions_tensor(expected[0], labels))
print(__ctc_decoder_predictions_tensor(produced[0], labels))    

28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	1
28	14
28	28
28	28
28	28
28	28
28	28
28	28
28	14
28	28
28	28
28	28
28	15
28	28
28	28
28	28
28	28
28	28
28	28
28	0
28	0
28	28
28	28
28	2
28	28
28	28
28	28
28	21
28	28
28	28
28	28
28	28
28	28
28	6
28	28
28	28
28	28
28	9
28	28
28	28
28	28
28	19
28	8
28	28
28	28
28	1
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	5
28	28
28	28
28	28
28	28
28	28
28	28
28	0
28	0
28	28
28	28
28	16
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
28	28
['']
['anno bufishae p']
